# Biostats280 Homework 1
- Author： Shuang Gao 
- Date: 2018/04/13

## Question 2
2.1 Associative rule for addition says (x + y) + z == x + (y + z). Check association rule using x = 0.1, y = 0.1 and z = 1.0 in Julia. Explain what you find.

In [49]:
x, y, z = 0.1, 0.1, 1.0
(x + y) + z == x + (y + z)

false

In [9]:
@show (x + y) + z
@show x + (y + z)

(x + y) + z = 1.2
x + (y + z) = 1.2000000000000002


1.2000000000000002

- The floating point in the left hand side is rounded to the first decimal place, but the floating point in the right hand side is arounded to 16th decimal place. Such rounding difference results in the violation of the associative rule for floating point addition.

2.2 Do floating-point numbers obey the associative rule for multiplication: (x * y) * z == x * (y * z)?

In [18]:
x, y, z = 0.3, 0.9, 0.4
(x * y) * z == x * (y * z)

true

In [19]:
@show (x * y) * z
@show x * (y * z)

(x * y) * z = 0.10800000000000001
x * (y * z) = 0.10800000000000001


0.10800000000000001

In [21]:
x, y, z = 0.3, 0.1, 0.4
(x * y) * z == x * (y * z)

false

In [22]:
@show (x * y) * z
@show x * (y * z)

(x * y) * z = 0.012
x * (y * z) = 0.012000000000000002


0.012000000000000002

- Same thing happens as in part 2.1. The floating point in the left hand side is rounded to the third decimal place, but the floating point in the right hand side is arounded to 18th decimal place. Such rounding difference results in the violation of the associative rule for floating point multiplication.

2.3 Do floating-point numbers obey the distributive rule: a * (x + y) == a * x + a * y?

In [39]:
a, x, y = 0.4, 0.1, 0.3 
a * (x + y) == a * x + a * y 

false

In [28]:
@show a * (x + y)
@show a * x + a * y 

a * (x + y) = 0.16000000000000003
a * x + a * y = 0.16


0.16

- The floating point in the left hand side is rounded to the 17th  decimal place, but the floating point in the right hand side is arounded to second decimal place. Such rounding difference results in the violation of the distributive rule for floating point.

2.4 Is 0 * x == 0 true for all floating-point number x?

In [41]:
x = Inf
0 * x == 0

false

In [43]:
@show 0 * x

0x = NaN


NaN

- The result of 0 * Inf returns NaN, which means not a number. Thus the comparision between NaN and 0 is not valid, which returns a false boolean result.

2.5 Is x / a == x * (1 / a) always true? Try with x = 3.0, a = 5.0 and explain what you find.

In [46]:
x, a = 3.0, 5.0
x / a == x * (1 / a)

false

In [48]:
@show x / a 
@show x * (1 / a)

x / a = 0.6
x * (1 / a) = 0.6000000000000001


0.6000000000000001

- The left hand side result is rounded to the first decimal place, but the right hand side result is arounded to 16th decimal place. Thus, the rounding difference of division may result in the inequality of x / a == x * (1 / a).

## Quesiton 3


In [99]:
function g(k)
    for i = 1:10
        k = 5k - 1
    end
    k
end


g (generic function with 1 method)

3.1 Use @code_llvm to find the LLVM bitcode of compiled g with Int64 input.

In [98]:
a = 2
typeof(a)

Int64

In [100]:
@code_llvm g(a)
g(a)


; Function Attrs: uwtable
define i64 @julia_g_64105(i64) #0 !dbg !5 {
top:
  %1 = mul i64 %0, 9765625
  %2 = add i64 %1, -2441406
  ret i64 %2
}


17089844

3.2 Use @code_llvm to find the LLVM bitcode of compiled g with Float64 input.

In [90]:
b = 2.0
typeof(b)

Float64

In [91]:
@code_llvm g(b)


; Function Attrs: uwtable
define double @julia_g_64099(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, 5.000000e+00
  %2 = fadd double %1, -1.000000e+00
  %3 = fmul double %2, 5.000000e+00
  %4 = fadd double %3, -1.000000e+00
  ret double %4
}


3.3 Compare the bitcode from questions 1 and 2. What do you find?

Let's reduce the loop in g function from 10 iterations to 3, and name it f, in order to find the way julia calculate.

In [92]:
function f(k)
    for i = 1:3
        k = 5k - 1
    end
    k
end


f (generic function with 1 method)

### Int64 input

In [93]:
# previous int64 input a
@code_llvm f(a)
f(a)


; Function Attrs: uwtable
define i64 @julia_f_64100(i64) #0 !dbg !5 {
top:
  %1 = mul i64 %0, 125
  %2 = add i64 %1, -31
  ret i64 %2
}


219

* From the output, julia attempts to calculate the result by modifying the equation $5k-1$ with associative and distributive rules first, and then plug in the unknown variable $k$. 
* Let's verify this idea with 3 iteration case:
$$
5(5(5k - 1) - 1) - 1 = 125k + (-25 -5 -1) \\
= 125k - 31
$$
125 and -31 are the results shown in `@code_llvm f(a)`. This modification can be done because integers obey associative and distributive rules in julia. 
* For function g, the calculation with Int64 input is modified to $9765625k-2441406$ with $k=a=2$, which shown in `@code_llvm g(a)`.

### Float64 input

In [101]:
# previous Float64 input b
@code_llvm f(b)
f(b)


; Function Attrs: uwtable
define double @julia_f_64103(double) #0 !dbg !5 {
top:
  %1 = fmul double %0, 5.000000e+00
  %2 = fadd double %1, -1.000000e+00
  %3 = fmul double %2, 5.000000e+00
  %4 = fadd double %3, -1.000000e+00
  %5 = fmul double %4, 5.000000e+00
  %6 = fadd double %5, -1.000000e+00
  ret double %6
}


219.0

* From the output, julia attempts to do the calculation following the iteration. For each iteration, the unknown $k$ is plugged in the equation $5k-1$. 
* Let's verify the idea with 3 iteration case:
 - For the first iteration, fraction multiplication is used for evaluatiing $5k-1$ with $k = 2.0$ and new $k=5k-1$ is stored. Thus, the parameters 5 and -1 are shown as floating points in `@code_llvm f(b)`.
 - For the second iteration, fraction multiplication is used for evaluatiing $5k-1$ with $k = 5*2.0-1$ and new $k=5k-1$ is stored. Thus, the parameters 5 and -1 are shown as floating points in `@code_llvm f(b)`.
 - For the third iteration, fraction multiplication is used for evaluatiing $5k-1$ with $k = 5*(5*2.0-1)-1$ and new $k=5k-1$ is stored. Thus, the parameters 5 and -1 are shown as floating points in `@code_llvm f(b)`.
